# Notebook te execute tests

In [1]:
%load_ext autoreload
%autoreload 2

# The import statements
from test_scripts import *
from experimentRunner import ExperimentRunner
from algorithms import *
from pathlib import Path
from metric_learn import *
from operator import itemgetter

def ignore_warnings():
    import warnings
    warnings.simplefilter(action='ignore', category=FutureWarning)
    warnings.simplefilter(action='ignore', category=Warning)

ignore_warnings()

In [2]:
# The setup for the time management
import datetime
currentDate = datetime.date.today()
currentMonthName = currentDate.strftime("%B")
currentDay = currentDate.strftime("%d-%m")

from datetime import datetime

start_year = '2022/09/26'

# convert string to date object
d1 = datetime.strptime(start_year, "%Y/%m/%d")
d2 = datetime.strptime(str(currentDate), "%Y-%m-%d")

# difference between dates in timedelta
delta = d2 - d1
#print(f'Difference is {delta.days} days')

nbWeek = int(delta.days/7) + 1

import os
from pathlib import Path
path = Path('experimenten/' + currentMonthName + '/week' + str(nbWeek)).absolute()
CHECK_FOLDER = os.path.isdir(path)
if not CHECK_FOLDER:
    os.makedirs(path)
    print("created folder : ", path)

else:
    print(path, "folder already exists.")

e:\Projecten\Unif_proj\ThesisCode\Testing\experimenten\November\week7 folder already exists.


In [3]:
# Dictionary with all the algorithms, datasets and preprocessing 

datasets = dict()
path_data = Path('datasets/cobras-paper/').absolute()
dir_list = os.listdir(path_data)
for i in range(len(dir_list)):
    datasets[i + 1] = dir_list[i][:len(dir_list[i]) - 5]
print(datasets)

algos = {
    0: Cobras(),
    1: SemiSupervised(),
    2: Supervised(),
    3: Preprocessed(),
    4: PortionPreprocessed()
}

ml_ss = {
    0: NCA(max_iter=100) # dees nog verder uitbereiden
}

ml_semi = {
    0: ITML()
}

{1: 'breast-cancer-wisconsin', 2: 'column_2C', 3: 'dermatology', 4: 'ecoli', 5: 'faces_expression_imagenet', 6: 'faces_eyes_imagenet', 7: 'faces_identity_imagenet', 8: 'faces_pose_imagenet', 9: 'glass', 10: 'hepatitis', 11: 'iris', 12: 'newsgroups_diff3', 13: 'newsgroups_sim3', 14: 'optdigits389_full', 15: 'parkinsons', 16: 'segmentation', 17: 'segmentation_training_only', 18: 'sonar', 19: 'spambase', 20: 'yeast'}


In [4]:
# Foldcreator

fold = 1
newFold = False

if newFold:
    print("Creating new fold to use in the experiments")
    for key, value in datasets.items():
        k = ExperimentRunner("Fold creator " + value , path, currentDay)
        fold = k.createFold(10, value)

## Effect of first training a metric on the trainingset or full set

In [4]:
# %%capture
k = ExperimentRunner("Effect_of_preprocessing_with_metric_learning", path, currentDay, saveResults= True) # nog ARI berekenen
k.loadDataSets([]) # list(itemgetter(11, 10)(datasets)) to get specific datasets
k.addAlgo(list(itemgetter(0,3, 4)(algos)))
k.run(runsPQ= 10, maxQ=200, crossFold=fold, save=True)
full = k.makePlot(maxQ = 200, sortByAlgo=True, sortByDataset=False, seperate=False)
seperate = k.makePlot(maxQ = 200, sortByAlgo=True, sortByDataset=False, seperate=True)
k.makeZip()

## Baseline

In [7]:
%%capture
k = ExperimentRunner("Baseline_SemiSupervised", path, currentDay, saveResults= True)
k.loadDataSets([]) # list(itemgetter(11, 10)(datasets)) to get specific datasets
k.addAlgo([Cobras(),BaselineSemiSupervised()])
k.run(runsPQ= 10, maxQ=200, crossFold=fold, save=True)
full = k.makePlot(maxQ = 100, sortByAlgo=True, sortByDataset=False, seperate=False)
seperate = k.makePlot(maxQ = 200, sortByAlgo=True, sortByDataset=False, seperate=True)
k.makeZip()

## Variance test

In [9]:
# variance test -> werkt gewoon met gemiddeldes I guess en geen crossfold
